# SOLUTIONS FOR PCA #10

# Second-order solutions to linear wave advection problem

This is an implementation of the linear wave advection problem pre-class assignment using the 2nd-order solutions (with two different slope limiters, which we implement for the in-class assignment!

Note that we go out of our way to set this up in a way where ghost zones, boundary conditions, etc. are taken care of cleanly because we want to make our lives easier down the line in the next class (when we solve Burger's equation).


In [ ]:
# User sets stuff in this cell

N = 128               # grid size (not including ghost zones)
u = 1.0               # wave speed, can be positive or negative
C = 0.8               # CFL, should be <= 1 for stability
N_periods = 1.0       # could be anything; 0.1 or 1.0 is a good choice for this assignment
IC_type = 'tophat'    # Initial condition: could be 'Gaussian' or 'tophat'
BC_type = 'periodic'  # Boundary condition: could be 'periodic' or 'Dirichlet' -- NOT IMPLEMENTED YET
adv_method  = '2oa'   # could be:  
                      #  1st order 'fou' or 'FTCS'
                      #  2nd order: '2oa' or '2oa_minmod' or '2oa_mcd'
save_figure = False   # Save auto-named plot to disk?  True or False

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# make a title and file name 

mytitle = "C = " + str(C) + ", Np = " + str(N_periods) + ", IC = " + IC_type + ", method = " + adv_method
myfilename = "C_" + str(C) + "_" + "Np_" + str(N_periods) + "_" + IC_type + "_" + adv_method + ".png"

print("TITLE:      ", mytitle)
print("FILE NAME:  ", myfilename)

# Grid dx
dx = 1.0/N

# final time (take abs. magnitude of velocity or else it'll be negative)
T_final = N_periods/np.abs(u)

# time step (need abs. magnitude of velocity or else it'll be negative)
dt = C*dx/np.abs(u)

In [ ]:
def ICs(N=100,IC_type='tophat',method='fou'):
    '''
    Initial condition generator.  Inputs are the desired number of grid 
    points (N) and initial conditions type (IC_type, options are 'tophat' 
    and 'Gaussian').  Also takes into account whether it's a 1st order method
    or second order method, which requires 1 or 2 ghost zones, respectively.
    
    Outputs are an array of positions (mostly useful for plotting) and
    the a values that are advected.
    '''
    
    if method == '2oa' or method == '2oa_minmod' or method == '2oa_mcd':
        # 2nd order schemes need 2 ghost zones
        ghost_zones = 2

    else:
        # 1st order schemes need 1 ghost zone
        ghost_zones = 1
    
    
    # 'a' array starts out zero everywhere.
    a = np.zeros(N+2*ghost_zones)
    
    dx=1.0/N
    
    # set up positions, including ghost zones, and also a step of 0.5*dx to make sure the points are at
    # the middle of the cell.
    x = np.linspace(0.0-dx*ghost_zones,1.0+dx*ghost_zones,num=N+2*ghost_zones,endpoint=False)+0.5*dx
    
    if IC_type=='tophat':  # top hat ICs
        
        # make a filter array that's True for the region we want 
        # to be 1, then set the array.
        filter_array = np.logical_and( x>0.35 , x<0.65)
        a[filter_array]=1.0

    elif IC_type=='Gaussian':  # Gaussian ICs
        sigma = 0.1
        a = np.exp(-(x-0.5)**2/(2*sigma*sigma))

    else:
        print("You didn't put in a correct IC type option:", IC_type)
    
    return x, a

In [ ]:
def copy_ghost_zones(array, method='fou'):
    '''
    Copies ghost zones for array.  If method is first order, copy only one.
    If method is second-order, copy two zones.
    
    AT SOME POINT this needs to be updated to also take into account 
    non-periodic boundary conditions.
    '''
    
    
    if method == '2oa' or method == '2oa_minmod' or method == '2oa_mcd':  
        # 2 ghost zones on each side for 2nd order methods
        array[0:2]=array[-4:-2]
        array[-2:]=array[2:4]
         
    else: 
        # 1 ghost zone on each side for 1st order methods
        array[0]=array[-2]
        array[-1]=array[1]

In [ ]:
def minmod(arr,dx):
    # minmod limiter - return 1st derivative, 
    # assume array has 1 extra zone on each side
    
    dadx = np.zeros(arr.size-2)
    
    for i in range(dadx.size):
        # the array indices here are all +1 more than they naively should be b/c of extra zones
        a = arr[i+1]-arr[i]
        b = arr[i+2]-arr[i+1]
        
        if np.abs(a) < np.abs(b) and a*b > 0:
            
            dadx[i] = a/dx

        elif np.abs(b) < np.abs(a) and a*b > 0:
        
            dadx[i] = b/dx

        else:
            dadx[i] = 0.0
            
    return dadx

def mcd(arr,dx):
    # monotonized central difference limiter (MCD) - return 1st derivative, 
    # assume array has 1 extra zone on each side
    
    dadx = np.zeros(arr.size-2)
    
    for i in range(dadx.size):
        # the array indices here are all +1 more than they naively should be b/c of extra zones
        a = arr[i+1]-arr[i]
        b = arr[i+2]-arr[i+1]
        c = arr[i+2]-arr[i]
        
        if a*b > 0.0:
            dadx[i] = np.fmin( np.fabs(c)/2.0, np.fmin( 2.0*np.fabs(b), 2.0*np.fabs(a) ))*np.sign(c)/dx
            
        else:
            dadx[i] = 0.0
            
    return dadx


In [ ]:
def advect(arr,u,dt,dx,n_grid,method='fou'):
    '''
    Advection routine.  Evolves the linear wave equation forward one timestep for various 1st
    and 2nd order methods (including 2nd order methods with minmod and mcd limiters).  
    
    Inputs are:
    
    arr = array that will be evolved
    u = velocity (can be positive or negative)
    dt = time step
    dx = grid spacing
    n_grid = number of grid points
    method = numerical method (1st order: 'fou' or 'FTCS', 2nd order: '2ou' or '2ou_minmod' or '2ou_mcd')
    
    '''
    
    if method=='fou':
        # first-order upwind
        
        if u > 0.0:
            arr[1:n_grid+1] = arr[1:n_grid+1] - u*(dt/dx)*(arr[1:n_grid+1]-arr[0:n_grid])           
        else:
            arr[1:n_grid+1] = arr[1:n_grid+1] - u*(dt/dx)*(arr[2:n_grid+2]-arr[1:n_grid+1])           
        
    elif method=='ftcs':
        # forward time center space (warning: garbage results ahead!)
        
        arr[1:n_grid+1] = arr[1:n_grid+1] - u*(dt/dx)*(arr[2:n_grid+2]-arr[0:n_grid])

    elif method=='2oa':
        # 2nd order advection: remember, two ghost zones!
        
        if u > 0.0:
            # use left solution for Riemann problem: plus signs.  Need cell i, i-1
            
            # i+1/2 uses values at cell i, and then i+1,i-1 for da/dx
            dadx = (arr[3:n_grid+3] - arr[1:n_grid+1])/(2.0*dx)
            aplus = arr[2:n_grid+2] + (dx/2.0)*(1-dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i-1, and then i, i-2 for da/dx
            dadx = (arr[2:n_grid+2] - arr[0:n_grid+0])/(2.0*dx)
            aminus = arr[1:n_grid+1] + (dx/2.0)*(1-dt*u/dx)*dadx
            
        else:
            # use right solution for Riemann problem: minus signs.  Need cell i, i+1
            
            # i+1/2 uses values at cell i+1, and then i+2,i for da/dx
            dadx = (arr[4:n_grid+4] - arr[2:n_grid+2])/(2.0*dx)
            aplus = arr[3:n_grid+3] - (dx/2.0)*(1+dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i, and then i+1, i-1 for da/dx
            dadx = (arr[3:n_grid+3] - arr[1:n_grid+1])/(2.0*dx)
            aminus = arr[2:n_grid+2] - (dx/2.0)*(1+dt*u/dx)*dadx
            
        arr[2:n_grid+2] = arr[2:n_grid+2] - u*(dt/dx)*(aplus-aminus)

    elif method=='2oa_minmod':
        # 2nd order advection: remember, two ghost zones!
        # this version uses the minmod limiter
        
        if u > 0.0:
            # use left solution for Riemann problem: plus signs.  Need cell i, i-1
            
            # i+1/2 uses values at cell i, and then i+1,i-1 for da/dx
            dadx = minmod(arr[1:-1],dx)
            aplus = arr[2:n_grid+2] + (dx/2.0)*(1-dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i-1, and then i, i-2 for da/dx
            dadx = minmod(arr[0:-2],dx)
            aminus = arr[1:n_grid+1] + (dx/2.0)*(1-dt*u/dx)*dadx
            
        else:
            # use right solution for Riemann problem: minus signs.  Need cell i, i+1
            
            # i+1/2 uses values at cell i+1, and then i+2,i for da/dx
            dadx = minmod(arr[2:],dx)
            aplus = arr[3:n_grid+3] - (dx/2.0)*(1+dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i, and then i+1, i-1 for da/dx
            dadx = minmod(arr[1:-1],dx)
            aminus = arr[2:n_grid+2] - (dx/2.0)*(1+dt*u/dx)*dadx
            
        arr[2:n_grid+2] = arr[2:n_grid+2] - u*(dt/dx)*(aplus-aminus)

    elif method=='2oa_mcd':
        # 2nd order advection: remember, two ghost zones!
        # this version uses the monotonized central difference limiter
        
        if u > 0.0:
            # use left solution for Riemann problem: plus signs.  Need cell i, i-1
            
            # i+1/2 uses values at cell i, and then i+1,i-1 for da/dx
            dadx = mcd(arr[1:-1],dx)
            aplus = arr[2:n_grid+2] + (dx/2.0)*(1-dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i-1, and then i, i-2 for da/dx
            dadx = mcd(arr[0:-2],dx)
            aminus = arr[1:n_grid+1] + (dx/2.0)*(1-dt*u/dx)*dadx
            
        else:
            # use right solution for Riemann problem: minus signs.  Need cell i, i+1
            
            # i+1/2 uses values at cell i+1, and then i+2,i for da/dx
            dadx = mcd(arr[2:],dx)
            aplus = arr[3:n_grid+3] - (dx/2.0)*(1+dt*u/dx)*dadx
            
            # i-1/2 uses values at cell i, and then i+1, i-1 for da/dx
            dadx = mcd(arr[1:-1],dx)
            aminus = arr[2:n_grid+2] - (dx/2.0)*(1+dt*u/dx)*dadx
            
        arr[2:n_grid+2] = arr[2:n_grid+2] - u*(dt/dx)*(aplus-aminus)
        
    else:
        print("not implemented!", method)


In [ ]:
def makeplot(x,a_orig,a_now,this_title='Output Information',filename='output.png',savefig=True):
    '''
    Makes a plot and (optionally) saves it to a file.  Inputs are:
    x = position array
    a_orig = original set of a values (initial conditions)
    a_now = current time set of a values
    this_title = your title
    filename = desired filename
    savefig = Boolean for whether or not you want your figure saved to a file.
    
    Output is a plot displayed to screen and (optionally) saved to file.
    '''
    plt.plot(x,a_orig,'k-')
    plt.plot(x,a_now,'r--')
    plt.xlabel('x')
    plt.ylabel('a(x)')
    plt.title(this_title)
    if savefig==True:
        plt.savefig(filename,dpi=400)


In [ ]:
# set up initial conditions arrays
x, a = ICs(N,IC_type,method=adv_method)

# make a copy of our initial conditions for plotting purposes
a_original = np.copy(a) 

# make a quick plot, just in case
plt.plot(x,a_original,'b-')
plt.xlabel('x')
plt.ylabel('a(x)')
plt.title('Initial conditions')

In [ ]:
# This cell evolves our simulation until the end time, using a timestep dt

# initial time is always zero
t=0.0

# loop until we're done.
# the -dt in the while() is because we add 
# the dt after we do the evolution.
# otherwise, things don't line up correctly.
while(t < T_final-dt):
        
    advect(a,u,dt,dx,N,method=adv_method)
    copy_ghost_zones(a,method=adv_method)
    
    t+=dt
    

In [ ]:
# make our plot and (optionally) save it.
makeplot(x,a_original,a,mytitle,myfilename,savefig=save_figure)
